<a href="https://colab.research.google.com/github/JamesAGV/Development-E/blob/main/opalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Desarrollo OPALO**

## **Instalación de paquetes**

In [6]:
#driver de comunicación con SQL Server
!pip install pyodbc 
#driver de comunicación S7-conncection
!pip install python-snap7 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.1 MB/s eta 0:00:00


## **Importación de librerías**

In [9]:
from sqlite3 import OperationalError
import pyodbc
import numpy as np
import snap7
from time import sleep
import csv
from datetime import datetime
import struct
import sys

## **Script**

In [14]:
class comm:
  def __init__(self, syn_offset_container, syn_offset_new_container, syn_offset_i_stack, db_data, bytes_buffer, offset_estado_plc, db_syn, server, database, username, password, ip_plc, rack, slot, tabla_rce, buffer_size, route_log, container_bytes):
      
    self.server = server
    self.database = database
    self.username = username
    self.password = password
    self.ip_plc = ip_plc
    self.rack = rack
    self.slot = slot
    self.db_syn = db_syn
    self.syn_offset_container = syn_offset_container
    self.syn_offset_new_container = syn_offset_new_container
    self.syn_offset_i_stack = syn_offset_i_stack
    self.container_bytes = container_bytes
    self.buffer_size = buffer_size
    self.db_data = db_data
    self.bytes_buffer = bytes_buffer
    self.tabla_rce = tabla_rce
    self.offset_estado_plc = offset_estado_plc
    self.route_log = route_log 

  def log(self, message):
    with open(self.route_log, 'a', newline='') as file:
      writer = csv.writer(file)
      writer.writerow([datetime.now(), message])

  def test_connection_plc(self):
    return self.client.get_cpu_state() != 'S7CpuStatusUnknown'

  def connect_plc(self):
    #self.log(message=f'Control {self.control}: Inicio de conexión con el PLC')
    self.client = snap7.client.Client()
    try:
      self.client.connect(self.ip_plc, self.rack, self.slot)
    except snap7.exceptions.Snap7Exception as e:
      # Error genérico de Snap7
      #pass
      self.log(message=f'Conexión fallida con PLC. Error: {e}')
    else:
      if self.test_connection_plc():
        self.log(message='Conexión exitosa con PLC')

  def disconnect_plc(self):
    self.client.disconnect()

  def reconnect_plc(self):
    while not self.test_connection_plc():
      sleep(1)
      self.log(message='Reconexión con PLC')
      self.connect_plc()    

  def read_bytes_from_plc(self, db, offset, num_bytes):     
    try:
      return self.client.db_read(db, offset, num_bytes)    
    except snap7.exceptions.Snap7Exception as e:
      self.log(message=f'Error leyendo bytes desde PLC: {e}')

  def write_bytes_to_plc(self, db, offset, bytes):
    try:
      self.client.db_write(db, offset, bytes)
    except snap7.exceptions.Snap7Exception as e:
      self.log(message=f'Error escribiendo bytes en el PLC: {e}')

  def get_new_container(self):
      return self.read_bytes_from_plc(db=self.db_syn, offset=self.syn_offset_new_container, num_bytes=1) == b'\x01'

  def set_new_container(self):
      self.write_bytes_to_plc(db=self.db_syn, offset=self.syn_offset_new_container, bytes=b'\x00')

  def get_container(self):
      bytes_container = self.read_bytes_from_plc(db=self.db_syn, offset=self.syn_offset_container, num_bytes=self.container_bytes)
      return int.from_bytes(bytes(bytes_container), byteorder='big', signed=False)

  def get_i_stack(self):
      i_stack_byte = self.read_bytes_from_plc(db=self.db_syn, offset=self.syn_offset_i_stack, num_bytes=1)
      i_stack = int.from_bytes(i_stack_byte, byteorder='big', signed=False)
      i_stack_next = (i_stack + 1) % self.buffer_size
      self.write_bytes_to_plc(db=self.db_syn, offset=self.syn_offset_i_stack, bytes=i_stack_next.to_bytes(1, sys.byteorder))   
      return i_stack

  def connect_database(self):
      try:
          self.con = pyodbc.connect('DRIVER={SQL Server};SERVER=' + self.server + ';DATABASE=' + self.database + ';UID=' + self.username + ';PWD=' + self.password)
          self.cursor = self.con.cursor()
          self.log(message='Conexión exitosa con la base de datos')
      except pyodbc.Error as e:
          self.log(message=f'Error de conexión con la base de datos: {e}')